# TESTING CODE

Model Used: CRF

Evalutation:
1. Precision: 0.873
2. Recall: 0.876     
3. F1 Score: 0.873 
4. Accuracy Score: 0.88

In [15]:
test_file = codecs.open('./final_test.txt', 'r', encoding="utf-8")
testinput = test_file.read()

In [16]:
X_test,y_test = prepareData(testinput)

In [35]:
# print(len(X_test))
# print(len(y_test))

In [27]:
def test_dev_data(X_test,y_test):
    crf = pickle.load(open('./annCorra_crf_all_features_model', 'rb'))
    y_pred=crf.predict(X_test)
    print("Accuracy")
    print(metrics.flat_accuracy_score(y_test, y_pred))
    print("F1 score on Test Data ")
    print(metrics.flat_f1_score(y_test, y_pred,average='weighted',labels=crf.classes_))
    print("F1 score on Training Data ")
    y_pred_train=crf.predict(X_train)
    print(metrics.flat_f1_score(y_train, y_pred_train,average='weighted',labels=crf.classes_))

    ### Look at class wise score
    print(metrics.flat_classification_report(
        y_test, y_pred, labels=crf.classes_, digits=3
    ))

In [26]:
test_dev_data(X_test, y_test)

Accuracy
0.8761872978384762
F1 score on Test Data 
0.8734304456337204
F1 score on Training Data 
0.9726559914522154
                precision    recall  f1-score   support

          ccof      0.686     0.682     0.684      2490
          rsym      0.999     1.000     0.999      2753
            r6      0.836     0.899     0.867      2287
      lwg__psp      0.999     0.999     0.999      8143
          nmod      0.635     0.562     0.596       698
     nmod__adj      0.933     0.961     0.947      3017
         r6-k2      0.612     0.534     0.570       277
           pof      0.858     0.879     0.869      1508
       pof__cn      1.000     1.000     1.000      2701
            k1      0.762     0.836     0.797      2302
           k7t      0.875     0.910     0.892       835
          main      0.695     0.749     0.721      1804
     lwg__vaux      0.999     0.999     0.999      1647
            k2      0.749     0.694     0.720      1926
            k7      0.834     0.814     0.8

# MODEL CODE

In [4]:
import re
import codecs
import pickle
from sklearn_crfsuite import CRF
from sklearn_crfsuite import metrics
from sklearn_crfsuite import scorers

In [5]:
def parse(input_):
    tags = []
    lexicons = []
    lemma = []
    pos = []
    tam = []
    gender = []
    number = []
    person = []
    sentences = input_.split("\n\n")
    for sentence in sentences:
        words = sentence.split("\n")
        for word in words:
            tokens = word.split("\t")
            try:
                tags.append(tokens[7])
                lexicons.append(tokens[1])
                pos.append(tokens[4])
                lemma.append(tokens[2])
                extra = tokens[5].split("|")
                for info in extra:
                    if "tam" in info:
                        tam_ = info.split("-")
                        tam.append(tam_[1])
                    if "gen" in info:
                        gen_ = info.split("-")
                        gender.append(gen_[1])
                    if "num" in info:
                        num_ = info.split("-")
                        number.append(num_[1])
                    if "pers" in info:
                        pers_ = info.split("-")
                        person.append(pers_[1])
            except:
                print(sentence)  
        lexicons.append("</s>")
        tags.append("</s>")
        lemma.append("</s>")
        pos.append("</s>")
        tam.append("</s>")
        gender.append("</s>")
        number.append("</s>")
        person.append("</s>")
    
    lexicons.pop()
    tags.pop()
    lemma.pop()
    pos.pop()
    tam.pop()
    gender.pop()
    number.pop()
    person.pop()
    return lexicons, tags, pos, lemma, tam, gender, number, person

In [6]:
def features(sentence, index, pos_sentence, tam, g, n ,p, lemma):
#     print(sentence[index], pos_sentence[index])
    ### sentence is of the form [w1,w2,w3,..], index is the position of the word in the sentence
    return {
        'word':sentence[index],
        'is_first_word': int(index==0),
        'is_last_word':int(index==len(sentence)-1),
        'prev_word':'' if index==0 else sentence[index-1],
        'next_word':'' if index==len(sentence)-1 else sentence[index+1],
        'prev_pre_word':'' if index==0 or index==1 else sentence[index-2],
        'next_next_word':'' if index==len(sentence)-1 or index==len(sentence)-2 else sentence[index+2],
        'is_numeric':int(sentence[index].isdigit()),
        'is_alphanumeric': int(bool((re.match('^(?=.*[0-9]$)(?=.*[a-zA-Z])',sentence[index])))),
        'pos': pos_sentence[index],
        'tam':tam[index],
        'gender': g[index],
        'number':n[index],
        'person':p[index],
        'lemma': lemma[index]
         }

In [7]:
def prepareData(input):
    lexicons, tags, pos, lemma, tam, g, n ,p = parse(input)
#     print(len(lexicons))
    sentences = ' '.join(lexicons).split(' </s> ')
    sentences_pos = ' '.join(pos).split(' </s> ')
    sentences_tags = ' '.join(tags).split(' </s> ')
    sentences_tam = ' '.join(tam).split(' </s> ')
    sentences_g = ' '.join(g).split(' </s> ')
    sentences_n = ' '.join(n).split(' </s> ')
    sentences_p = ' '.join(p).split(' </s> ')
    sentences_lemma = ' '.join(lemma).split(' </s> ')
#     print(sentences_tags)
#     print(len(sentences))
    X=[]
    y=[]
    for sentenceid, sentence in enumerate(sentences):
        words = sentence.split(' ')
        pos = sentences_pos[sentenceid].split(' ')
        tam = sentences_tam[sentenceid].split(' ')
        g = sentences_g[sentenceid].split(' ')
        n = sentences_n[sentenceid].split(' ')
        p = sentences_p[sentenceid].split(' ')
        lemma = sentences_lemma[sentenceid].split(' ')
        X.append([features(words, index, pos, tam, g, n ,p, lemma) for index in range(len(words))])
    for sentence_tag in sentences_tags:
        words_tag = sentence_tag.split(' ')
        y.append(words_tag)
    return X, y

In [8]:
def train_CRF(X_train,y_train):   
    crf = CRF(
        algorithm='lbfgs',
        c1=0.01,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=True
    )
    crf.fit(X_train, y_train)
    pickle.dump(crf, open("./annCorra_crf_all_features_model", 'wb'))

In [9]:
train_file = codecs.open('./final_train.txt', 'r', encoding="utf-8")
traininput = train_file.read()

# dev_file = codecs.open('./final_dev.txt', 'r', encoding="utf-8")
# devinput = dev_file.read()

In [10]:
X_train, y_train = prepareData(traininput)

In [13]:
# print((X_train[0]))
# print((y_train))

In [14]:
train_CRF(X_train, y_train)

In [38]:
# X_dev,y_dev = prepareData(devinput)

In [39]:
# test_dev_data(X_test, y_test)